In [1]:
import csv
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt

### Part 1: Data Pre - Processing

In [88]:
dat_orig = pd.read_csv('zip_codes_and_income.csv', low_memory = False)

In [3]:
pd.options.display.max_columns = 300

In [4]:
dat_orig.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10,HD01_VD11,HD02_VD11,HD01_VD12,HD02_VD12,HD01_VD13,HD02_VD13,HD01_VD14,HD02_VD14,HD01_VD15,HD02_VD15,HD01_VD16,HD02_VD16,HD01_VD17,HD02_VD17,HD01_VD18,HD02_VD18,HD01_VD19,HD02_VD19,HD01_VD20,HD02_VD20,HD01_VD21,HD02_VD21,HD01_VD22,HD02_VD22,HD01_VD23,HD02_VD23,HD01_VD24,HD02_VD24,HD01_VD25,HD02_VD25,HD01_VD26,HD02_VD26,HD01_VD27,HD02_VD27,HD01_VD28,HD02_VD28,HD01_VD29,HD02_VD29,HD01_VD30,HD02_VD30,HD01_VD31,HD02_VD31,HD01_VD32,HD02_VD32,HD01_VD33,HD02_VD33,HD01_VD34,HD02_VD34,HD01_VD35,HD02_VD35,HD01_VD36,HD02_VD36,HD01_VD37,HD02_VD37,HD01_VD38,HD02_VD38,HD01_VD39,HD02_VD39,HD01_VD40,HD02_VD40,HD01_VD41,HD02_VD41,HD01_VD42,HD02_VD42,HD01_VD43,HD02_VD43,HD01_VD44,HD02_VD44,HD01_VD45,HD02_VD45,HD01_VD46,HD02_VD46,HD01_VD47,HD02_VD47,HD01_VD48,HD02_VD48,HD01_VD49,HD02_VD49,HD01_VD50,HD02_VD50,HD01_VD51,HD02_VD51,HD01_VD52,HD02_VD52,HD01_VD53,HD02_VD53,HD01_VD54,HD02_VD54,HD01_VD55,HD02_VD55,HD01_VD56,HD02_VD56,HD01_VD57,HD02_VD57,HD01_VD58,HD02_VD58,HD01_VD59,HD02_VD59,HD01_VD60,HD02_VD60,HD01_VD61,HD02_VD61,HD01_VD62,HD02_VD62,HD01_VD63,HD02_VD63,HD01_VD64,HD02_VD64,HD01_VD65,HD02_VD65,HD01_VD66,HD02_VD66,HD01_VD67,HD02_VD67,HD01_VD68,HD02_VD68,HD01_VD69,HD02_VD69
0,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,Estimate; Householder under 25 years:,Margin of Error; Householder under 25 years:,Estimate; Householder under 25 years: - Less t...,Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $10,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $15,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $20,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $25,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $30,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $35,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $40,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $45,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $50,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $60,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $75,00...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $100,0...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $125,0...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $150,0...",Margin of Error; Householder under 25 years: -...,"Estimate; Householder under 25 years: - $200,0...",Margin of Error; Householder under 25 years: -...,Estimate; Householder 25 to 44 years:,Margin of Error; Householder 25 to 44 years:,Estimate; Householder 25 to 44 years: - Less t...,Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $10,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $15,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $20,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $25,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $30,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Householder 25 to 44 years: - $35,00...",Margin of Error; Householder 25 to 44 years: -...,"Estimate; Househo

In [5]:
dat_orig.shape

(33121, 141)

This data set was taken from us census data and can be found [here](https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_17_5YR_B19037&prodType=table). Filter for age of householder, household earning, and 5 digit zips.

The first step is to trim the dataset to only include the estimate columns (drop the margin of errors.)

In [8]:
dat_trim = dat_orig[['GEO.id2', 'HD01_VD01','HD01_VD02','HD01_VD03','HD01_VD04','HD01_VD05','HD01_VD06','HD01_VD07','HD01_VD08','HD01_VD09','HD01_VD10','HD01_VD11','HD01_VD12','HD01_VD13','HD01_VD14','HD01_VD15','HD01_VD16','HD01_VD17','HD01_VD18','HD01_VD19','HD01_VD20','HD01_VD21','HD01_VD22','HD01_VD23','HD01_VD24','HD01_VD25','HD01_VD26','HD01_VD27','HD01_VD28','HD01_VD29','HD01_VD30','HD01_VD31','HD01_VD32','HD01_VD33','HD01_VD34','HD01_VD35','HD01_VD36','HD01_VD37','HD01_VD38','HD01_VD39','HD01_VD40','HD01_VD41','HD01_VD42','HD01_VD43','HD01_VD44','HD01_VD45','HD01_VD46','HD01_VD47','HD01_VD48','HD01_VD49','HD01_VD50','HD01_VD51','HD01_VD52','HD01_VD53','HD01_VD54','HD01_VD55','HD01_VD56','HD01_VD57','HD01_VD58','HD01_VD59','HD01_VD60','HD01_VD61','HD01_VD62','HD01_VD63','HD01_VD64','HD01_VD65','HD01_VD66','HD01_VD67','HD01_VD68','HD01_VD69']]
dat_trim.head()

,GEO.id2,HD01_VD01,HD01_VD02,HD01_VD03,HD01_VD04,HD01_VD05,HD01_VD06,HD01_VD07,HD01_VD08,HD01_VD09,HD01_VD10,HD01_VD11,HD01_VD12,HD01_VD13,HD01_VD14,HD01_VD15,HD01_VD16,HD01_VD17,HD01_VD18,HD01_VD19,HD01_VD20,HD01_VD21,HD01_VD22,HD01_VD23,HD01_VD24,HD01_VD25,HD01_VD26,HD01_VD27,HD01_VD28,HD01_VD29,HD01_VD30,HD01_VD31,HD01_VD32,HD01_VD33,HD01_VD34,HD01_VD35,HD01_VD36,HD01_VD37,HD01_VD38,HD01_VD39,HD01_VD40,HD01_VD41,HD01_VD42,HD01_VD43,HD01_VD44,HD01_VD45,HD01_VD46,HD01_VD47,HD01_VD48,HD01_VD49,HD01_VD50,HD01_VD51,HD01_VD52,HD01_VD53,HD01_VD54,HD01_VD55,HD01_VD56,HD01_VD57,HD01_VD58,HD01_VD59,HD01_VD60,HD01_VD61,HD01_VD62,HD01_VD63,HD01_VD64,HD01_VD65,HD01_VD66,HD01_VD67,HD01_VD68,HD01_VD69
0,Id2,Estimate; Total:,Estimate; Householder under 25 years:,Estimate; Householder under 25 years: - Less t...,"Estimate; Householder under 25 years: - $10,00...","Estimate; Householder under 25 years: - $15,00...","Estimate; Householder under 25 years: - $20,00...","Estimate; Householder under 25 years: - $25,00...","Estimate; Householder under 25 years: - $30,00...","Estimate; Householder under 25 years: - $35,00...","Estimate; Householder under 25 years: - $40,00...","Estimate; Householder under 25 years: - $45,00...","Estimate; Householder under 25 years: - $50,00...","Estimate; Householder under 25 years: - $60,00...","Estimate; Householder under 25 years: - $75,00...","Estimate; Householder under 25 years: - $100,0...","Estimate; Householder under 25 years: - $125,0...","Estimate; Householder under 25 years: - $150,0...","Estimate; Householder under 25 years: - $200,0...",Estimate; Householder 25 to 44 years:,Estimate; Householder 25 to 44 years: - Less t...,"Estimate; Householder 25 to 44 years: - $10,00...","Estimate; Householder 25 to 44 years: - $15,00...","Estimate; Householder 25 to 44 years: - $20,00...","Estimate; Householder 25 to 44 years: - $25,00...","Estimate; Householder 25 to 44 years: - $30,00...","Estimate; Householder 25 to 44 years: - $35,00...","Estimate; Householder 25 to 44 years: - $40,00...","Estimate; Householder 25 to 44 years: - $45,00...","Estimate; Householder 25 to 44 years: - $50,00...","Estimate; Householder 25 to 44 years: - $60,00...","Estimate; Householder 25 to 44 years: - $75,00...","Estimate; Householder 25 to 44 years: - $100,0...","Estimate; Householder 25 to 44 years: - $125,0...","Estimate; Householder 25 to 44 years: - $150,0...","Estimate; Householder 25 to 44 years: - $200,0...",Estimate; Householder 45 to 64 years:,Estimate; Householder 45 to 64 years: - Less t...,"Estimate; Householder 45 to 64 years: - $10,00...","Estimate; Householder 45 to 64 years: - $15,00...","Estimate; Householder 45 to 64 years: - $20,00...","Estimate; Householder 45 to 64 years: - $25,00...","Estimate; Householder 45 to 64 years: - $30,00...","Estimate; Householder 45 to 64 years: - $35,00...","Estimate; Householder 45 to 64 years: - $40,00...","Estimate; Householder 45 to 64 years: - $45,00...","Estimate; Householder 45 to 64 years: - $50,00...","Estimate; Householder 45 to 64 years: - $60,00...","Estimate; Householder 45 to 64 years: - $75,00...","Estimate; Householder 45 to 64 years: - $100,0...","Estimate; Householder 45 to 64 years: - $125,0...","Estimate; Householder 45 to 64 years: - $150,0...","Estimate; Householder 45 to 64 years: - $200,0...",Estimate; Householder 65 years and over:,Estimate; Householder 65 years and over: - Les...,Estimate; Householder 65 years and over: - $10...,Estimate; Householder 65 years and over: - $15...,Estimate; Householder 65 years and over: - $20...,Estimate; Householder 65 years and over: - $25...,Estimate; Householder 65 years and over: - $30...,Estimate; Householder 65 years and over: - $35...,Estimate; Householder 65 years and over: - $40...,Estimate; Householder 65 years and over: - $45...,Estimate; Householder 65 years and over: - $50...,Estimate; Householder 65 years and over: - $60...,Estimate; Householder 65 years and over: - $75...,Estimate; Householder 65 years an

Column names need fixing. We'll make the first row (row 0) be our new columns names.

In [9]:
dat_trim.columns = dat_trim.iloc[0]
dat_trim = dat_trim.reindex(dat_trim.index.drop(0))
dat_trim.head()

,Id2,Estimate; Total:,Estimate; Householder under 25 years:,"Estimate; Householder under 25 years: - Less than $10,000","Estimate; Householder under 25 years: - $10,000 to $14,999","Estimate; Householder under 25 years: - $15,000 to $19,999","Estimate; Householder under 25 years: - $20,000 to $24,999","Estimate; Householder under 25 years: - $25,000 to $29,999","Estimate; Householder under 25 years: - $30,000 to $34,999","Estimate; Householder under 25 years: - $35,000 to $39,999","Estimate; Householder under 25 years: - $40,000 to $44,999","Estimate; Householder under 25 years: - $45,000 to $49,999","Estimate; Householder under 25 years: - $50,000 to $59,999","Estimate; Householder under 25 years: - $60,000 to $74,999","Estimate; Householder under 25 years: - $75,000 to $99,999","Estimate; Householder under 25 years: - $100,000 to $124,999","Estimate; Householder under 25 years: - $125,000 to $149,999","Estimate; Householder under 25 years: - $150,000 to $199,999","Estimate; Householder under 25 years: - $200,000 or more",Estimate; Householder 25 to 44 years:,"Estimate; Householder 25 to 44 years: - Less than $10,000","Estimate; Householder 25 to 44 years: - $10,000 to $14,999","Estimate; Householder 25 to 44 years: - $15,000 to $19,999","Estimate; Householder 25 to 44 years: - $20,000 to $24,999","Estimate; Householder 25 to 44 years: - $25,000 to $29,999","Estimate; Householder 25 to 44 years: - $30,000 to $34,999","Estimate; Householder 25 to 44 years: - $35,000 to $39,999","Estimate; Householder 25 to 44 years: - $40,000 to $44,999","Estimate; Householder 25 to 44 years: - $45,000 to $49,999","Estimate; Householder 25 to 44 years: - $50,000 to $59,999","Estimate; Householder 25 to 44 years: - $60,000 to $74,999","Estimate; Householder 25 to 44 years: - $75,000 to $99,999","Estimate; Householder 25 to 44 years: - $100,000 to $124,999","Estimate; Householder 25 to 44 years: - $125,000 to $149,999","Estimate; Householder 25 to 44 years: - $150,000 to $199,999","Estimate; Householder 25 to 44 years: - $200,000 or more",Estimate; Householder 45 to 64 years:,"Estimate; Householder 45 to 64 years: - Less than $10,000","Estimate; Householder 45 to 64 years: - $10,000 to $14,999","Estimate; Householder 45 to 64 years: - $15,000 to $19,999","Estimate; Householder 45 to 64 years: - $20,000 to $24,999","Estimate; Householder 45 to 64 years: - $25,000 to $29,999","Estimate; Householder 45 to 64 years: - $30,000 to $34,999","Estimate; Householder 45 to 64 years: - $35,000 to $39,999","Estimate; Householder 45 to 64 years: - $40,000 to $44,999","Estimate; Householder 45 to 64 years: - $45,000 to $49,999","Estimate; Householder 45 to 64 years: - $50,000 to $59,999","Estimate; Householder 45 to 64 years: - $60,000 to $74,999","Estimate; Householder 45 to 64 years: - $75,000 to $99,999","Estimate; Householder 45 to 64 years: - $100,000 to $124,999","Estimate; Householder 45 to 64 years: - $125,000 to $149,999","Estimate; Householder 45 to 64 years: - $150,000 to $199,999","Estimate; Householder 45 to 64 years: - $200,000 or more",Estimate; Householder 65 years and over:,"Estimate; Householder 65 years and over: - Less than $10,000","Estimate; Householder 65 years and over: - $10,000 to $14,999","Estimate; Householder 65 years and over: - $15,000 to $19,999","Estimate; Householder 65 years and over: - $20,000 to $24,999","Estimate; Householder 65 years and over: - $25,000 to $29,999","Estimate; Householder 65 years and over: - $30,000 to $34,999","Estimate; Householder 65 years and over: - $35,000 to $39,999","Estimate; Householder 65 years and over: - $40,000 to $44,999","Estimate; Householder 65 years and over: - $45,000 to $49,999","Estimate; Householder 65 years and over: - $50,000 to $59,999","Estimate; Householder 65 years and over: - $60,000 to $74,999","Estimate; Householder 65 years and over: - $75,000 to $99,999","Estimate; Householder 65 years and over: - $100,000 to $124,999","Estimate; Householder 65 years and over: - $125,000 

In [11]:
dat_trim.shape

(33120, 70)

We have some columns counting the totals overall and by age group. We'll want to drop these as we're going to be agging the data granularly, so having the totals included will mess up the way we'll eventually be summing the rows. First we identify these "total: columns:

In [180]:
dat_trim.iloc[:, np.r_[1:3, 19:20, 36:37, 53:54]].head()

,Estimate; Total:,Estimate; Householder under 25 years:,Estimate; Householder 25 to 44 years:,Estimate; Householder 45 to 64 years:,Estimate; Householder 65 years and over:
1,5818,23,1513,2540,1742
2,12719,99,2775,5908,3937
3,19009,513,5229,6885,6382
4,1959,9,418,855,677
5,9120,138,2142,3396,3444


Before we drop the totals, let's first sum the total columns so we know how many households we had in the original, untouched data. Then as we build our new data set we can use this as a benchmark to verify that we aren't accidentally dropping useful data.

In [13]:
dat_trim["Estimate; Total:"] = dat_trim["Estimate; Total:"].astype('int64', copy=False)
dat_trim["Estimate; Total:"].sum()

120046688

We'll drop these columns from the data set we're going to be building, but we'll keep these totals so we can sanity check ourselves when we've built a melted, long version of this wide data set.

In [14]:
totals = [1,2,19,36,53]
dat_trim.drop(dat_trim.columns[totals],axis=1,inplace=True)
dat_trim.head()

,Id2,"Estimate; Householder under 25 years: - Less than $10,000","Estimate; Householder under 25 years: - $10,000 to $14,999","Estimate; Householder under 25 years: - $15,000 to $19,999","Estimate; Householder under 25 years: - $20,000 to $24,999","Estimate; Householder under 25 years: - $25,000 to $29,999","Estimate; Householder under 25 years: - $30,000 to $34,999","Estimate; Householder under 25 years: - $35,000 to $39,999","Estimate; Householder under 25 years: - $40,000 to $44,999","Estimate; Householder under 25 years: - $45,000 to $49,999","Estimate; Householder under 25 years: - $50,000 to $59,999","Estimate; Householder under 25 years: - $60,000 to $74,999","Estimate; Householder under 25 years: - $75,000 to $99,999","Estimate; Householder under 25 years: - $100,000 to $124,999","Estimate; Householder under 25 years: - $125,000 to $149,999","Estimate; Householder under 25 years: - $150,000 to $199,999","Estimate; Householder under 25 years: - $200,000 or more","Estimate; Householder 25 to 44 years: - Less than $10,000","Estimate; Householder 25 to 44 years: - $10,000 to $14,999","Estimate; Householder 25 to 44 years: - $15,000 to $19,999","Estimate; Householder 25 to 44 years: - $20,000 to $24,999","Estimate; Householder 25 to 44 years: - $25,000 to $29,999","Estimate; Householder 25 to 44 years: - $30,000 to $34,999","Estimate; Householder 25 to 44 years: - $35,000 to $39,999","Estimate; Householder 25 to 44 years: - $40,000 to $44,999","Estimate; Householder 25 to 44 years: - $45,000 to $49,999","Estimate; Householder 25 to 44 years: - $50,000 to $59,999","Estimate; Householder 25 to 44 years: - $60,000 to $74,999","Estimate; Householder 25 to 44 years: - $75,000 to $99,999","Estimate; Householder 25 to 44 years: - $100,000 to $124,999","Estimate; Householder 25 to 44 years: - $125,000 to $149,999","Estimate; Householder 25 to 44 years: - $150,000 to $199,999","Estimate; Householder 25 to 44 years: - $200,000 or more","Estimate; Householder 45 to 64 years: - Less than $10,000","Estimate; Householder 45 to 64 years: - $10,000 to $14,999","Estimate; Householder 45 to 64 years: - $15,000 to $19,999","Estimate; Householder 45 to 64 years: - $20,000 to $24,999","Estimate; Householder 45 to 64 years: - $25,000 to $29,999","Estimate; Householder 45 to 64 years: - $30,000 to $34,999","Estimate; Householder 45 to 64 years: - $35,000 to $39,999","Estimate; Householder 45 to 64 years: - $40,000 to $44,999","Estimate; Householder 45 to 64 years: - $45,000 to $49,999","Estimate; Householder 45 to 64 years: - $50,000 to $59,999","Estimate; Householder 45 to 64 years: - $60,000 to $74,999","Estimate; Householder 45 to 64 years: - $75,000 to $99,999","Estimate; Householder 45 to 64 years: - $100,000 to $124,999","Estimate; Householder 45 to 64 years: - $125,000 to $149,999","Estimate; Householder 45 to 64 years: - $150,000 to $199,999","Estimate; Householder 45 to 64 years: - $200,000 or more","Estimate; Householder 65 years and over: - Less than $10,000","Estimate; Householder 65 years and over: - $10,000 to $14,999","Estimate; Householder 65 years and over: - $15,000 to $19,999","Estimate; Householder 65 years and over: - $20,000 to $24,999","Estimate; Householder 65 years and over: - $25,000 to $29,999","Estimate; Householder 65 years and over: - $30,000 to $34,999","Estimate; Householder 65 years and over: - $35,000 to $39,999","Estimate; Householder 65 years and over: - $40,000 to $44,999","Estimate; Householder 65 years and over: - $45,000 to $49,999","Estimate; Householder 65 years and over: - $50,000 to $59,999","Estimate; Householder 65 years and over: - $60,000 to $74,999","Estimate; Householder 65 years and over: - $75,000 to $99,999","Estimate; Householder 65 years and over: - $100,000 to $124,999","Estimate; Householder 65 years and over: - $125,000 to $149,999","Estimate; Householder 65 years and over: - $150,000 to $199,999","Estimate; Householder 65 years and over: - $200,000 or more"
1,601,8,0,15,0,0,0,0,0,0,0,0,0,

In [15]:
dat_trim.shape

(33120, 65)

We want to flip this data set from wide (65 columns) to long (just a column for age, income, and count).

In [16]:
# let the zip code be the id, and all the income buckets will get flattened into one column
# we'll run the function once for each age bucket, where an age bucket spans columns a to b
def melt_data(dat, a, b, age_bracket):
    new_dat = dat.iloc[:, np.r_[0:1, a:b]]
    stack = pd.melt(new_dat, id_vars=['Id2'], value_vars=list(new_dat)[1:], var_name='income_bucket', value_name='count')
    stack['age_bracket'] = age_bracket
    return stack

In [154]:
dat_trim.head()

,Id2,"Estimate; Householder under 25 years: - Less than $10,000","Estimate; Householder under 25 years: - $10,000 to $14,999","Estimate; Householder under 25 years: - $15,000 to $19,999","Estimate; Householder under 25 years: - $20,000 to $24,999","Estimate; Householder under 25 years: - $25,000 to $29,999","Estimate; Householder under 25 years: - $30,000 to $34,999","Estimate; Householder under 25 years: - $35,000 to $39,999","Estimate; Householder under 25 years: - $40,000 to $44,999","Estimate; Householder under 25 years: - $45,000 to $49,999","Estimate; Householder under 25 years: - $50,000 to $59,999","Estimate; Householder under 25 years: - $60,000 to $74,999","Estimate; Householder under 25 years: - $75,000 to $99,999","Estimate; Householder under 25 years: - $100,000 to $124,999","Estimate; Householder under 25 years: - $125,000 to $149,999","Estimate; Householder under 25 years: - $150,000 to $199,999","Estimate; Householder under 25 years: - $200,000 or more","Estimate; Householder 25 to 44 years: - Less than $10,000","Estimate; Householder 25 to 44 years: - $10,000 to $14,999","Estimate; Householder 25 to 44 years: - $15,000 to $19,999","Estimate; Householder 25 to 44 years: - $20,000 to $24,999","Estimate; Householder 25 to 44 years: - $25,000 to $29,999","Estimate; Householder 25 to 44 years: - $30,000 to $34,999","Estimate; Householder 25 to 44 years: - $35,000 to $39,999","Estimate; Householder 25 to 44 years: - $40,000 to $44,999","Estimate; Householder 25 to 44 years: - $45,000 to $49,999","Estimate; Householder 25 to 44 years: - $50,000 to $59,999","Estimate; Householder 25 to 44 years: - $60,000 to $74,999","Estimate; Householder 25 to 44 years: - $75,000 to $99,999","Estimate; Householder 25 to 44 years: - $100,000 to $124,999","Estimate; Householder 25 to 44 years: - $125,000 to $149,999","Estimate; Householder 25 to 44 years: - $150,000 to $199,999","Estimate; Householder 25 to 44 years: - $200,000 or more","Estimate; Householder 45 to 64 years: - Less than $10,000","Estimate; Householder 45 to 64 years: - $10,000 to $14,999","Estimate; Householder 45 to 64 years: - $15,000 to $19,999","Estimate; Householder 45 to 64 years: - $20,000 to $24,999","Estimate; Householder 45 to 64 years: - $25,000 to $29,999","Estimate; Householder 45 to 64 years: - $30,000 to $34,999","Estimate; Householder 45 to 64 years: - $35,000 to $39,999","Estimate; Householder 45 to 64 years: - $40,000 to $44,999","Estimate; Householder 45 to 64 years: - $45,000 to $49,999","Estimate; Householder 45 to 64 years: - $50,000 to $59,999","Estimate; Householder 45 to 64 years: - $60,000 to $74,999","Estimate; Householder 45 to 64 years: - $75,000 to $99,999","Estimate; Householder 45 to 64 years: - $100,000 to $124,999","Estimate; Householder 45 to 64 years: - $125,000 to $149,999","Estimate; Householder 45 to 64 years: - $150,000 to $199,999","Estimate; Householder 45 to 64 years: - $200,000 or more","Estimate; Householder 65 years and over: - Less than $10,000","Estimate; Householder 65 years and over: - $10,000 to $14,999","Estimate; Householder 65 years and over: - $15,000 to $19,999","Estimate; Householder 65 years and over: - $20,000 to $24,999","Estimate; Householder 65 years and over: - $25,000 to $29,999","Estimate; Householder 65 years and over: - $30,000 to $34,999","Estimate; Householder 65 years and over: - $35,000 to $39,999","Estimate; Householder 65 years and over: - $40,000 to $44,999","Estimate; Householder 65 years and over: - $45,000 to $49,999","Estimate; Householder 65 years and over: - $50,000 to $59,999","Estimate; Householder 65 years and over: - $60,000 to $74,999","Estimate; Householder 65 years and over: - $75,000 to $99,999","Estimate; Householder 65 years and over: - $100,000 to $124,999","Estimate; Householder 65 years and over: - $125,000 to $149,999","Estimate; Householder 65 years and over: - $150,000 to $199,999","Estimate; Householder 65 years and over: - $200,000 or more"
1,601,8,0,15,0,0,0,0,0,0,0,0,0,

In [17]:
under_25 = melt_data(dat_trim, 1, 17, 'under 25')
under_45 = melt_data(dat_trim, 17, 33, '25 to 44')
under_65 = melt_data(dat_trim, 33, 49, '45 to 65')
over_65 = melt_data(dat_trim, 49, 65, '65 +')


In [19]:
bind = pd.concat([under_25, under_45, under_65, over_65])
bind.head()

,Id2,income_bucket,count,age_bracket
0,601,Estimate; Householder under 25 years: - Less t...,8,under 25
1,602,Estimate; Householder under 25 years: - Less t...,71,under 25
2,603,Estimate; Householder under 25 years: - Less t...,354,under 25
3,606,Estimate; Householder under 25 years: - Less t...,9,under 25
4,610,Estimate; Householder under 25 years: - Less t...,60,under 25


The income bucket rows are wordy, we can trim everything preceding the dash to make the data more digestible.

In [20]:
bind['income_bucket'] = bind['income_bucket'].apply(lambda x: x.split('- ',1)[1])
bind.head()

,Id2,income_bucket,count,age_bracket
0,601,"Less than $10,000",8,under 25
1,602,"Less than $10,000",71,under 25
2,603,"Less than $10,000",354,under 25
3,606,"Less than $10,000",9,under 25
4,610,"Less than $10,000",60,under 25


Next we verify that we still have all of our households.

In [21]:
bind['count'] = bind['count'].astype('int64', copy=False)
bind['count'].sum()

120046688

As a sanity check, we see this total sums to the same number we got when summing the "Estimate Total" column from the original data set. 
This confirms that we built our long data set correctly so we can move forward.

As a sanity check, we see there are roughly 120 million households in our new, long dataset. Now, we can filter the data by age bracket, income bucket, and zip code. The next steps will be to filter these households to ones that contain women by household type (married, single mom, single woman), add a finer level of granularity along the age facet, and finally we will join in a zip code data set to gather additional geographical and regional information (state, city, region, division etc).

### Part 2: Merging and Multiplying

The US Census provides additonal data on household types. Family vs non-family. Family house holds can be either heterosexual married couples are single parents with children. Non family households can be homosexual couples without kids or single men/women. The data sets can be found [here](https://www.census.gov/data/tables/time-series/demo/income-poverty/cps-hinc/hinc-04.html).
This data is taken from 2017 and cites 127 million households. The zip code data set we loaded from the american fact finder cited 120 million households. The discrepancy is reasonable given that the american fact finder uses a five year estimate (thus averages the past 5 years to get a rolling 5 year average for each year). We will be using the 2017 census data as a means for understanding the percentage distribution of households that contain women, so the discrepancy in counts shouldn't affect the end result. The methodology used was to gather 5 separate data sets: married-couple family households, female house holder (no spouse), male house holder (no spouse), nonfamily female householder, and nonfamily male householder.
Then, of the total count of households within each bucket we assumed that only married households or female headed-households contain. The result gleaned from slicing the data this way is that 12.4% of households are either single dads with children or male householders with no spouse present. If we multiply all our household counts by .876, then we retain the count of households which contain women.

In [40]:
bind['contains_woman'] = bind['count']
bind['contains_woman'] = bind['contains_woman'].astype(float)
bind['contains_woman'] = bind['contains_woman']*.876
bind.head()

,Id2,income_bucket,count,age_bracket,contains_woman
0,601,"Less than $10,000",8,under 25,7.008
1,602,"Less than $10,000",71,under 25,62.196
2,603,"Less than $10,000",354,under 25,310.104
3,606,"Less than $10,000",9,under 25,7.884
4,610,"Less than $10,000",60,under 25,52.560


In [41]:
bind['contains_woman'].sum()

105160898.68799996

We retain 105 million of the total 120 million households in the United States as households which contain at least 1 woman. 
Note that according the 2017 US Census there are 162 million women [living in the US](https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_17_5YR_S0101&prodType=table)
If we assume that only these 105 households contain those 162 million women, then we assume on average 1.54 women per household.
Note that some married couples may live with a mother or grown daughter. Also some single women heads of households may live 
with a friend or roommate, and some female heads of households here are in gay relationships. Before we dig too deep into this
though let's first subset our households to those buckets where the householder age is over 45.

In [42]:
bind.age_bracket.unique()

array(['under 25', '25 to 44', '45 to 65', '65 +'], dtype=object)

In [43]:
good_ages = ['45 to 65', '65 +']

In [44]:
bind.income_bucket.unique()

array(['Less than $10,000', '$10,000 to $14,999', '$15,000 to $19,999',
       '$20,000 to $24,999', '$25,000 to $29,999', '$30,000 to $34,999',
       '$35,000 to $39,999', '$40,000 to $44,999', '$45,000 to $49,999',
       '$50,000 to $59,999', '$60,000 to $74,999', '$75,000 to $99,999',
       '$100,000 to $124,999', '$125,000 to $149,999',
       '$150,000 to $199,999', '$200,000 or more'], dtype=object)

In [45]:
good_incomes = ['$75,000 to $99,999', '$100,000 to $124,999', '$125,000 to $149,999',
       '$150,000 to $199,999', '$200,000 or more']

Subset the merged data to only the ages and incomes we care about

In [48]:
trimmed = bind[bind['age_bracket'].isin(good_ages)]
trimmed['count'].sum()

76406080

Here we see there are 76.4 million households where the age of the householder is in the qualifying age range for J.Jill. 

In [49]:
trimmed.contains_woman.sum()

66931726.07999998

We also see that of the 76.4 million age qualifying households, approximately 66.9 million contain a woman 
(using our 87.6% estimator).
Looking at US Census data, there are 69,631,000 females in the United States over the age 45. This implies that within 
our 87.6% estimator may have been slightly too conservative, but is generally in the right ballpark. 
Next we will filter our 76 million households further to those that qualify along our income facet.

In [50]:
trimmed = trimmed[trimmed['income_bucket'].isin(good_incomes)]

In [37]:
trimmed['count'] = trimmed['count'].astype('int64', copy=False)
trimmed['count'].sum()

29234330

In [51]:
trimmed.contains_woman.sum()

25609273.08000001

We retain 29.2 million of the 76 million age-qualifying householders when we restrict household income to above 75k. Of the 29.2
million age and income qualifying households, an estimated 25.6 million of them contain a woman.
Our next task is to obtain slightly wider age buckets for this data. We can do this by using the same census data we used
to obtain the statistic that 87.6% of all U.S. households contain at least one woman.

In [52]:
trimmed.head()

,Id2,income_bucket,count,age_bracket,contains_woman
364320,601,"$75,000 to $99,999",19,45 to 65,16.644
364321,602,"$75,000 to $99,999",191,45 to 65,167.316
364322,603,"$75,000 to $99,999",219,45 to 65,191.844
364323,606,"$75,000 to $99,999",0,45 to 65,0.000
364324,610,"$75,000 to $99,999",127,45 to 65,111.252


According to the census data subsetted to age and income qualifying households, 83.8% of our target market is a married-couple 
household, 8.08% of are single mother households, and 8.12% are single women. We can multiply our `contains_woman` 
column by these percentages to get household counts along the household type facet.
*Note that we are assuming household types are distributed evenly across age groups and geographical region*.

To do this we will create three data sets and row bind them together. We will create add a new column `household_type` to 
identify the household class.

In [54]:
married = {'counts':trimmed['contains_woman']*.838, 'zip_code': trimmed.Id2, 'income_bucket': trimmed.income_bucket, 'age_bucket': trimmed.age_bracket}
married = pd.DataFrame(married)
married['household_type'] = 'married'

single_mom = {'counts':trimmed['contains_woman']*.0808, 'zip_code': trimmed.Id2, 'income_bucket': trimmed.income_bucket, 'age_bucket': trimmed.age_bracket}
single_mom = pd.DataFrame(single_mom)
single_mom['household_type'] = 'single_mom'

single_woman = {'counts':trimmed['contains_woman']*.0812, 'zip_code': trimmed.Id2, 'income_bucket': trimmed.income_bucket, 'age_bucket': trimmed.age_bracket}
single_woman = pd.DataFrame(single_woman)
single_woman['household_type'] = 'single_woman'

In [55]:
married.head()

,counts,zip_code,income_bucket,age_bucket,household_type
364320,13.947672,601,"$75,000 to $99,999",45 to 65,married
364321,140.210808,602,"$75,000 to $99,999",45 to 65,married
364322,160.765272,603,"$75,000 to $99,999",45 to 65,married
364323,0.000000,606,"$75,000 to $99,999",45 to 65,married
364324,93.229176,610,"$75,000 to $99,999",45 to 65,married


In [56]:
married.shape

(331200, 5)

In [57]:
trimmed.head()

,Id2,income_bucket,count,age_bracket,contains_woman
364320,601,"$75,000 to $99,999",19,45 to 65,16.644
364321,602,"$75,000 to $99,999",191,45 to 65,167.316
364322,603,"$75,000 to $99,999",219,45 to 65,191.844
364323,606,"$75,000 to $99,999",0,45 to 65,0.000
364324,610,"$75,000 to $99,999",127,45 to 65,111.252


In [58]:
trimmed['contains_woman'].sum()

25609273.08000001

In [59]:
married.counts.sum() + single_woman.counts.sum() + single_mom.counts.sum()

25609273.08000001

Here we verified that new data sets with household specific counts still sums to the same 25.6 million household total. So we
can safely bind them together into one new data frame.

In [60]:
households = pd.concat([married, single_woman, single_mom])

In [61]:
households.head()

,counts,zip_code,income_bucket,age_bucket,household_type
364320,13.947672,601,"$75,000 to $99,999",45 to 65,married
364321,140.210808,602,"$75,000 to $99,999",45 to 65,married
364322,160.765272,603,"$75,000 to $99,999",45 to 65,married
364323,0.000000,606,"$75,000 to $99,999",45 to 65,married
364324,93.229176,610,"$75,000 to $99,999",45 to 65,married


In [62]:
households.counts.sum()

25609273.08000002

Next part is tricky. We have decent income buckets and zip codes and household types (we know there is at least one woman). 
But our age buckets are fairly wide. We will apply percentages now to get the data in a from that has narrow age buckets.
The percentages were calculated using the census data taken [here](https://www.census.gov/data/tables/time-series/demo/income-poverty/cps-hinc/hinc-02.html).
The methodology was to gather all 5 household types and the counts within each 5 year age bracket along qualifying age and
income ranges. Then, for each household type, the sum of the households within the 45 - 65 and 65+ age range was calculated. Then,
the percentage of households within each age range was calculated for the available 5 year age segments. To estimate the household
counts along these narrower age ranges, we need only multiply our household counts by the respective age bucket percentage. This
approach is outlined below.

In [63]:
married.tail()

,counts,zip_code,income_bucket,age_bucket,household_type
529915,0.000000,99923,"$200,000 or more",65 +,married
529916,0.000000,99925,"$200,000 or more",65 +,married
529917,0.000000,99926,"$200,000 or more",65 +,married
529918,0.000000,99927,"$200,000 or more",65 +,married
529919,0.734088,99929,"$200,000 or more",65 +,married


for each zip code, we will multiply the count by a percentage to get a _new_ count for a _narrower_ age bucket

In [64]:
married1 = married.query('age_bucket == "45 to 65"').copy()
married2 = married.query('age_bucket == "65 +"').copy()

married1['40 - 44'] = married1.counts * .192
married1['45 - 49'] = married1.counts * .202
married1['50 - 54'] = married1.counts * .197
married1['55 - 59'] = married1.counts * .214
married1['60 - 64'] = married1.counts * .195

married2['65 - 69'] = married2.counts * .385
married2['70 - 74'] = married2.counts * .284
married2['75+'] = married2.counts * .331



single_mom1 = single_mom.query('age_bucket == "45 to 65"').copy()
single_mom2 = single_mom.query('age_bucket == "65 +"').copy()

single_mom1['40 - 44'] = single_mom1.counts * .24
single_mom1['45 - 49'] = single_mom1.counts * .24
single_mom1['50 - 54'] = single_mom1.counts * .21
single_mom1['55 - 59'] = single_mom1.counts * .18
single_mom1['60 - 64'] = single_mom1.counts * .13

single_mom2['65 - 69'] = single_mom2.counts * .32
single_mom2['70 - 74'] = single_mom2.counts * .24
single_mom2['75+'] = single_mom2.counts * .44



single_woman1 = single_woman.query('age_bucket == "45 to 65"').copy()
single_woman2 = single_woman.query('age_bucket == "65 +"').copy()

single_woman1['40 - 44'] = single_woman1.counts * .11
single_woman1['45 - 49'] = single_woman1.counts * .14
single_woman1['50 - 54'] = single_woman1.counts * .20
single_woman1['55 - 59'] = single_woman1.counts * .27
single_woman1['60 - 64'] = single_woman1.counts * .28

single_woman2['65 - 69'] = single_woman2.counts * .23
single_woman2['70 - 74'] = single_woman2.counts * .22
single_woman2['75+'] = single_woman2.counts * .55






In [65]:
married1.head()

,counts,zip_code,income_bucket,age_bucket,household_type,40 - 44,45 - 49,50 - 54,55 - 59,60 - 64
364320,13.947672,601,"$75,000 to $99,999",45 to 65,married,2.677953,2.817430,2.747691,2.984802,2.719796
364321,140.210808,602,"$75,000 to $99,999",45 to 65,married,26.920475,28.322583,27.621529,30.005113,27.341108
364322,160.765272,603,"$75,000 to $99,999",45 to 65,married,30.866932,32.474585,31.670759,34.403768,31.349228
364323,0.000000,606,"$75,000 to $99,999",45 to 65,married,0.000000,0.000000,0.000000,0.000000,0.000000
364324,93.229176,610,"$75,000 to $99,999",45 to 65,married,17.900002,18.832294,18.366148,19.951044,18.179689


In [66]:
test = pd.melt(married1, id_vars=['zip_code', 'income_bucket', 'household_type', 'age_bucket', 'counts'], value_vars=['40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64'], var_name='narrow_age_bucket', value_name='narrow_count')
test.query('zip_code == "601" and income_bucket == "$75,000 to $99,999"').head(100)

,zip_code,income_bucket,household_type,age_bucket,counts,narrow_age_bucket,narrow_count
0,601,"$75,000 to $99,999",married,45 to 65,13.947672,40 - 44,2.677953
165600,601,"$75,000 to $99,999",married,45 to 65,13.947672,45 - 49,2.817430
331200,601,"$75,000 to $99,999",married,45 to 65,13.947672,50 - 54,2.747691
496800,601,"$75,000 to $99,999",married,45 to 65,13.947672,55 - 59,2.984802
662400,601,"$75,000 to $99,999",married,45 to 65,13.947672,60 - 64,2.719796


Now we can be happy we melted the data correctly, so we can drop the `counts` columns (as this is just a total). Maybe we'll keep the wide age bucket for now in case we want it for any reason.

Next we just need to stack all the data together into one happy family.

In [67]:
m1 = pd.melt(married1, id_vars=['zip_code', 'income_bucket', 'household_type', 'age_bucket', 'counts'], value_vars=['40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64'], var_name='narrow_age_bucket', value_name='narrow_count')
sm1 = pd.melt(single_mom1, id_vars=['zip_code', 'income_bucket', 'household_type', 'age_bucket', 'counts'], value_vars=['40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64'], var_name='narrow_age_bucket', value_name='narrow_count')
sw1 = pd.melt(single_woman1, id_vars=['zip_code', 'income_bucket', 'household_type', 'age_bucket', 'counts'], value_vars=['40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64'], var_name='narrow_age_bucket', value_name='narrow_count')

In [68]:
single_mom2.head()

,counts,zip_code,income_bucket,age_bucket,household_type,65 - 69,70 - 74,75+
364320,0.353904,601,"$75,000 to $99,999",65 +,single_mom,0.113249,0.084937,0.155718
364321,7.431984,602,"$75,000 to $99,999",65 +,single_mom,2.378235,1.783676,3.270073
364322,10.829462,603,"$75,000 to $99,999",65 +,single_mom,3.465428,2.599071,4.764963
364323,0.000000,606,"$75,000 to $99,999",65 +,single_mom,0.000000,0.000000,0.000000
364324,0.000000,610,"$75,000 to $99,999",65 +,single_mom,0.000000,0.000000,0.000000


In [69]:
m2 = pd.melt(married2, id_vars=['zip_code', 'income_bucket', 'household_type', 'age_bucket', 'counts'], value_vars=['65 - 69', '70 - 74', '75+'], var_name='narrow_age_bucket', value_name='narrow_count')
sm2 = pd.melt(single_mom2, id_vars=['zip_code', 'income_bucket', 'household_type', 'age_bucket', 'counts'], value_vars=['65 - 69', '70 - 74', '75+'], var_name='narrow_age_bucket', value_name='narrow_count')
sw2 = pd.melt(single_woman2, id_vars=['zip_code', 'income_bucket', 'household_type', 'age_bucket', 'counts'], value_vars=['65 - 69', '70 - 74', '75+'], var_name='narrow_age_bucket', value_name='narrow_count')

In [70]:
final_data_set = pd.concat([m1, m2, sm1, sm2, sw1, sw2])

In [71]:
final_data_set.head()

,zip_code,income_bucket,household_type,age_bucket,counts,narrow_age_bucket,narrow_count
0,601,"$75,000 to $99,999",married,45 to 65,13.947672,40 - 44,2.677953
1,602,"$75,000 to $99,999",married,45 to 65,140.210808,40 - 44,26.920475
2,603,"$75,000 to $99,999",married,45 to 65,160.765272,40 - 44,30.866932
3,606,"$75,000 to $99,999",married,45 to 65,0.000000,40 - 44,0.000000
4,610,"$75,000 to $99,999",married,45 to 65,93.229176,40 - 44,17.900002


In [72]:
final_data_set.narrow_count.sum()

25609273.080000028

Next we'll join in some additional zip code data to gain further information from these zip codes (state, region, division, city etc)

In [73]:
uszips = pd.read_csv('uszips.csv')
uszips.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone
0,601,18.1800,-66.7522,Adjuntas,PR,Puerto Rico,True,NaN,18570,111.4,72001,Adjuntas,"{'72001':99.43,'72141':0.57}",False,False,America/Puerto_Rico
1,602,18.3607,-67.1752,Aguada,PR,Puerto Rico,True,NaN,41520,523.7,72003,Aguada,{'72003':100},False,False,America/Puerto_Rico
2,603,18.4544,-67.1220,Aguadilla,PR,Puerto Rico,True,NaN,54689,667.9,72005,Aguadilla,{'72005':100},False,False,America/Puerto_Rico
3,606,18.1672,-66.9383,Maricao,PR,Puerto Rico,True,NaN,6615,60.4,72093,Maricao,"{'72093':94.88,'72121':1.35,'72153':3.78}",False,False,America/Puerto_Rico
4,610,18.2903,-67.1224,Anasco,PR,Puerto Rico,True,NaN,29016,311.9,72011,Añasco,"{'72003':0.55,'72011':99.45}",False,False,America/Puerto_Rico


In [74]:
len(list(uszips.zip.unique()))

33099

In [75]:
len(list(final_data_set.zip_code.unique()))

33120

In [76]:
final_data_set.shape

(3974400, 7)

In [77]:
uszips.shape

(33099, 16)

In [78]:
final_data_set.dtypes

zip_code              object
income_bucket         object
household_type        object
age_bucket            object
counts               float64
narrow_age_bucket     object
narrow_count         float64
dtype: object

In [79]:
final_data_set['zip_code']  = final_data_set['zip_code'].astype(str).astype(int)

In [80]:
dat = pd.merge(final_data_set, uszips, left_on = 'zip_code', right_on = 'zip', how = 'left')

In [81]:
dat.head()

,zip_code,income_bucket,household_type,age_bucket,counts,narrow_age_bucket,narrow_count,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone
0,601,"$75,000 to $99,999",married,45 to 65,13.947672,40 - 44,2.677953,601.0,18.1800,-66.7522,Adjuntas,PR,Puerto Rico,True,NaN,18570.0,111.4,72001.0,Adjuntas,"{'72001':99.43,'72141':0.57}",False,False,America/Puerto_Rico
1,602,"$75,000 to $99,999",married,45 to 65,140.210808,40 - 44,26.920475,602.0,18.3607,-67.1752,Aguada,PR,Puerto Rico,True,NaN,41520.0,523.7,72003.0,Aguada,{'72003':100},False,False,America/Puerto_Rico
2,603,"$75,000 to $99,999",married,45 to 65,160.765272,40 - 44,30.866932,603.0,18.4544,-67.1220,Aguadilla,PR,Puerto Rico,True,NaN,54689.0,667.9,72005.0,Aguadilla,{'72005':100},False,False,America/Puerto_Rico
3,606,"$75,000 to $99,999",married,45 to 65,0.000000,40 - 44,0.000000,606.0,18.1672,-66.9383,Maricao,PR,Puerto Rico,True,NaN,6615.0,60.4,72093.0,Maricao,"{'72093':94.88,'72121':1.35,'72153':3.78}",False,False,America/Puerto_Rico
4,610,"$75,000 to $99,999",married,45 to 65,93.229176,40 - 44,17.900002,610.0,18.2903,-67.1224,Anasco,PR,Puerto Rico,True,NaN,29016.0,311.9,72011.0,Añasco,"{'72003':0.55,'72011':99.45}",False,False,America/Puerto_Rico


In [82]:
len(list(dat.state_id.unique()))

53

In [83]:
regions = pd.read_csv('regions.csv')
regions.head()

,state,region,division,state_abr
0,Connecticut,Northeast,New England,CT
1,Maine,Northeast,New England,ME
2,Massachusetts,Northeast,New England,MA
3,New Hampshire,Northeast,New England,NH
4,Rhode Island,Northeast,New England,RI


In [84]:
dat['state_id']  = dat['state_id'].astype(str)
regions['state_abr']  = regions['state_abr'].astype(str)
dat_r = pd.merge(dat, regions, left_on = 'state_id', right_on = 'state_abr', how = 'left')
dat_r.head()

,zip_code,income_bucket,household_type,age_bucket,counts,narrow_age_bucket,narrow_count,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone,state,region,division,state_abr
0,601,"$75,000 to $99,999",married,45 to 65,13.947672,40 - 44,2.677953,601.0,18.1800,-66.7522,Adjuntas,PR,Puerto Rico,True,NaN,18570.0,111.4,72001.0,Adjuntas,"{'72001':99.43,'72141':0.57}",False,False,America/Puerto_Rico,NaN,NaN,NaN,NaN
1,602,"$75,000 to $99,999",married,45 to 65,140.210808,40 - 44,26.920475,602.0,18.3607,-67.1752,Aguada,PR,Puerto Rico,True,NaN,41520.0,523.7,72003.0,Aguada,{'72003':100},False,False,America/Puerto_Rico,NaN,NaN,NaN,NaN
2,603,"$75,000 to $99,999",married,45 to 65,160.765272,40 - 44,30.866932,603.0,18.4544,-67.1220,Aguadilla,PR,Puerto Rico,True,NaN,54689.0,667.9,72005.0,Aguadilla,{'72005':100},False,False,America/Puerto_Rico,NaN,NaN,NaN,NaN
3,606,"$75,000 to $99,999",married,45 to 65,0.000000,40 - 44,0.000000,606.0,18.1672,-66.9383,Maricao,PR,Puerto Rico,True,NaN,6615.0,60.4,72093.0,Maricao,"{'72093':94.88,'72121':1.35,'72153':3.78}",False,False,America/Puerto_Rico,NaN,NaN,NaN,NaN
4,610,"$75,000 to $99,999",married,45 to 65,93.229176,40 - 44,17.900002,610.0,18.2903,-67.1224,Anasco,PR,Puerto Rico,True,NaN,29016.0,311.9,72011.0,Añasco,"{'72003':0.55,'72011':99.45}",False,False,America/Puerto_Rico,NaN,NaN,NaN,NaN


In [85]:
dat_r.tail()

,zip_code,income_bucket,household_type,age_bucket,counts,narrow_age_bucket,narrow_count,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone,state,region,division,state_abr
3974395,99923,"$200,000 or more",single_woman,65 +,0.000000,75+,0.000000,99923.0,55.9811,-130.0405,Hyder,AK,Alaska,True,NaN,87.0,2.4,2198.0,Prince of Wales-Hyder,{'02198':100},False,False,America/Sitka,Alaska,West,Pacific,AK
3974396,99925,"$200,000 or more",single_woman,65 +,0.000000,75+,0.000000,99925.0,55.5577,-132.9763,Klawock,AK,Alaska,True,NaN,819.0,5.7,2198.0,Prince of Wales-Hyder,{'02198':100},False,False,America/Sitka,Alaska,West,Pacific,AK
3974397,99926,"$200,000 or more",single_woman,65 +,0.000000,75+,0.000000,99926.0,55.1262,-131.4893,Metlakatla,AK,Alaska,True,NaN,1460.0,4.2,2198.0,Prince of Wales-Hyder,{'02198':100},False,False,America/Metlakatla,Alaska,West,Pacific,AK
3974398,99927,"$200,000 or more",single_woman,65 +,0.000000,75+,0.000000,99927.0,56.2510,-133.3757,Point Baker,AK,Alaska,True,NaN,94.0,0.2,2198.0,Prince of Wales-Hyder,{'02198':100},False,False,America/Sitka,Alaska,West,Pacific,AK
3974399,99929,"$200,000 or more",single_woman,65 +,0.071131,75+,0.039122,99929.0,56.3670,-131.9329,Wrangell,AK,Alaska,True,NaN,2338.0,0.4,2275.0,Wrangell,{'02275':100},False,False,America/Sitka,Alaska,West,Pacific,AK


In [86]:
dat_r.shape

(3974400, 27)

In [87]:
dat_r.narrow_count.sum()

25609273.080000028

In [90]:
dat_r.to_csv("market_analysis.csv",index=False)